In [1]:
#pip install pandas langchain openai tiktoken

from langchain_core.documents import Document

In [2]:
import pandas as pd
from pathlib import Path

# 1. 입력 CSV 경로들
CSV_FILES = [
    "data/processed/law/law_labeled.csv",
    "data/processed/rule/rule_labeled.csv",
    "data/processed/case/case_labeled.csv",
]

# 2. CSV 로드
dfs = []
for path in CSV_FILES:
    df = pd.read_csv(path)
    dfs.append(df)

# 3. 하나로 병합
merged_df = pd.concat(dfs, ignore_index=True)

# 4. (선택) id 중복 체크
dup_ids = merged_df["id"].duplicated().sum()
print(f"중복 id 개수: {dup_ids}")

# 5. 저장
output_path = Path("data/processed/all_chunks.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)

merged_df.to_csv(output_path, index=False)

print(f"✅ 병합 완료: {output_path}")
print(f"총 청크 수: {len(merged_df)}")

중복 id 개수: 0
✅ 병합 완료: data\processed\all_chunks.csv
총 청크 수: 248


In [3]:
import pandas as pd
from langchain_core.documents import Document

def load_chunked_csv(csv_path: str):
    df = pd.read_csv(csv_path)

    documents = []
    for _, row in df.iterrows():
        documents.append(
            Document(
                page_content=row["text"],
                metadata={
                    "id": row["id"],
                    "category": row["category"],
                    "priority": int(row["priority"]),
                    "article": row["article"],
                    "chunk_id": row["chunk_id"],
                    "source": row["source"],
                }
            )
        )
    return documents

In [4]:
docs = load_chunked_csv("data/processed/all_chunks.csv")

In [5]:
docs[0]

Document(metadata={'id': 'ea20858b-3e6b-4fab-b297-a185d0f34d1f', 'category': 'law', 'priority': 4, 'article': '제390조', 'chunk_id': '본문', 'source': '민법(법률)(제20432호)(20260101)-간략.docx'}, page_content='(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.\n제2장 계약\n제6절 사용대차')

In [6]:
docs[0].page_content

'(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.\n제2장 계약\n제6절 사용대차'

In [7]:
docs[0].page_content[:300]

'(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.\n제2장 계약\n제6절 사용대차'

In [8]:
for i in range(5):
    print(f"\n--- doc {i} ---")
    print(docs[i].metadata)
    print(docs[i].page_content[:200])


--- doc 0 ---
{'id': 'ea20858b-3e6b-4fab-b297-a185d0f34d1f', 'category': 'law', 'priority': 4, 'article': '제390조', 'chunk_id': '본문', 'source': '민법(법률)(제20432호)(20260101)-간략.docx'}
(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.
제2장 계약
제6절 사용대차

--- doc 1 ---
{'id': '767cb115-6be6-441b-a624-0de4fdd5d05d', 'category': 'law', 'priority': 4, 'article': '제618조', 'chunk_id': '본문', 'source': '민법(법률)(제20432호)(20260101)-간략.docx'}
(임대차의 의의) 임대차는 당사자 일방이 상대방에게 목적물을 사용, 수익하게 할 것을 약정하고 상대방이 이에 대하여 차임을 지급할 것을 약정함으로써 그 효력이 생긴다.

--- doc 2 ---
{'id': '1bb029e6-f430-4d8e-8777-f93e7ac57c90', 'category': 'law', 'priority': 4, 'article': '제619조', 'chunk_id': '본문', 'source': '민법(법률)(제20432호)(20260101)-간략.docx'}
(처분능력, 권한없는 자의 할 수 있는 단기임대차) 처분의 능력 또는 권한없는 자가 임대차를 하는 경우에는 그 임대차는 다음 각호의 기간을 넘지 못한다.
1. 식목, 채염 또는 석조, 석회조, 연와조 및 이와 유사한 건축을 목적으로 한 토지의 임대차는 10년
2. 기타 토지의 임대차는 5년
3. 건물 기타 공작물의 임대차는 3년
4. 동산의 임대차는 6월

--- doc 3 ---
{'id': '1527dfbd-1069-

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [10]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

texts = [doc.page_content for doc in docs]

vectors = embeddings.embed_documents(texts)

print(f"임베딩 완료: {len(vectors)}")
print(f"벡터 차원: {len(vectors[0])}")

임베딩 완료: 248
벡터 차원: 3072


In [13]:
import numpy as np

np.save(
    "data/processed/law_vectors.npy",
    np.array(vectors)
)

In [ ]:
import json

metadata = [doc.metadata for doc in docs]

with open(
    "data/embeddings/law_metadata.json",
    "w",
    encoding="utf-8"
) as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

In [32]:
embedded_items = []

for doc, vector in zip(docs, vectors):
    embedded_items.append({
        "id": doc.metadata["id"],
        "vector": vector,
        "text": doc.page_content,
        "metadata": doc.metadata
    })

embedded_items[0]

{'id': 'ea20858b-3e6b-4fab-b297-a185d0f34d1f',
 'vector': [0.0010057338513433933,
  -0.045739274471998215,
  -0.00908906664699316,
  0.025705287232995033,
  -0.006449376232922077,
  0.020714081823825836,
  -0.034765537828207016,
  -0.004195840563625097,
  -0.0379008911550045,
  -0.014789185486733913,
  0.021451812237501144,
  0.0018861111020669341,
  -0.016103267669677734,
  -0.015596078708767891,
  -0.008195721544325352,
  -0.02731907367706299,
  -0.020495068281888962,
  0.016368389129638672,
  -0.021163636818528175,
  0.00535142607986927,
  -0.038269754499197006,
  0.024990612640976906,
  0.012898752465844154,
  -0.006795187015086412,
  -0.03723232448101044,
  0.008357100188732147,
  0.0011699940077960491,
  0.018846701830625534,
  -0.0003431093937251717,
  -0.02902507409453392,
  -0.02900202013552189,
  0.006339868996292353,
  -0.04094402864575386,
  -0.006679916754364967,
  0.006835531909018755,
  -0.025128936395049095,
  -0.009204337373375893,
  -0.020391324535012245,
  -0.0090775

In [37]:
pinecone_vectors = [
    {
        "id": item["id"],
        "values": item["vector"],
        "metadata": item["metadata"]
    }
    for item in embedded_items
]

# PINECONE


In [39]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

index = pc.Index("realestate")  # 네 인덱스 이름

In [40]:
pinecone_vectors = []

for item in embedded_items:
    pinecone_vectors.append({
        "id": item["id"],
        "values": item["vector"],
        "metadata": item["metadata"]
    })

In [41]:
BATCH_SIZE = 100

for i in range(0, len(pinecone_vectors), BATCH_SIZE):
    batch = pinecone_vectors[i:i + BATCH_SIZE]
    index.upsert(vectors=batch)
    print(f"{i} ~ {i + len(batch)} 업로드 완료")

0 ~ 100 업로드 완료
100 ~ 200 업로드 완료
200 ~ 248 업로드 완료


In [42]:
res = index.query(
    vector=embedded_items[0]["vector"],
    top_k=5,
    include_metadata=True
)

for m in res["matches"]:
    print(m["score"], m["metadata"].get("article"))

0.999982834 제390조
0.49426651 제6조의2
0.460227966 사례 1
0.451234847 제617조
0.585020065 제750조


# Retriever 붙이기 (LangChain)